In [102]:
from openai import OpenAI
import pandas as pd
import numpy as np
import anthropic
import json
import os
import re
import time
from dotenv import load_dotenv, find_dotenv
import requests
from mistralai import Mistral
import google.generativeai as genai

The goal of this sheet is to test three different questions and their categorizations holding all else constant. I will use 6 categories of questions. We will use 5 different models:

1. GPT4
2. Google (Palm or Gemini)
3. Claude 3
4. Mistral (open-source)
5. Llama3 (open-source)

Observations:

1. Openai GPT4 is fast and consistent
2. Claude is slower, but also consistent
3. Llama3 was the most difficult to implement, slower, and most-error prone (lots of errors caused me an extra hour of work)
4. Mistral was fast, easy to implement, and no errors
5. Google was fast, but repeated errors to the API calls and had to add wait times in between calls. It looks like google does not want its API to be used for these types of purposes. Failed with status code: 429. I ended up getting locked out of making API calls and was not able to complete the categorization. 

error rates:
a19i
gpt4 - 0
claude - 0
llama - 22
mistral - 0

the 8B parameter models were quick but large error rates.

In [107]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

below we are setting the values for the GPT models

In [111]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4o-2024-11-20"
GPT_reasoning = "o1-preview"
GPT_reasoning_mini = "o1-mini"
claude = "claude-3-5-sonnet-20241022"
perplexity = "llama-3.1-sonar-large-128k-online"
perplexity_light = "llama-3.1-sonar-small-128k-online"
perplexity_heavy ="llama-3.1-sonar-huge-128k-online"
mistral = "mistral-large-latest" #version 24.11
mistral_light = "ministral-8b-latest" #version 24.10
google = "gemini-1.5-pro"
google_light = "gemini-1.5-flash"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [5]:
column1 = "a19i"
column2 = "a19f"
column3 = "e1b"

question1 = "Why did you move?"
question2 = "After this last move, what steps, if any, did you take in order to make new friends?"
question3 = "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"

user_categories1 = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard"]

user_categories2 = ["Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.",
                   "Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.",
                   "Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.",
                   "Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.",
                   "Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.",
                   "Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs."]

user_categories3 = ["A simple statement without further elaboration, often expressed simply as 'because I know they would help'",
                   "Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.",
                   "Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.",
                   "Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).",
                   "Mention of a family member's personal character, such as 'they are a generous person' or 'they aren't good people'.",
                   "Past instances of receiving help or support from family members"]

In [6]:
UCNets_a19i = pd.read_excel("/Users/chrissoria/Documents/Research/UCNets_Classification/data/Raw_Cond_for_Coding_all_waves.xlsx", engine='openpyxl',sheet_name="JOINT_DATA",usecols=[column1])
UCNets_a19f = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19fg/a19fg_Master.xlsx", engine='openpyxl', sheet_name="master_a19f")
UCNets_e1b = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1ab_Master.xlsx", engine='openpyxl', sheet_name="Master_coded")

UCNets_a19f = UCNets_a19f[['Response']]
UCNets_a19f.rename(columns={'Response': 'a19f'}, inplace=True)

UCNets_e1b = UCNets_e1b[['Response']]
UCNets_e1b.rename(columns={'Response': 'e1b'}, inplace=True)


UCNets_a19i = UCNets_a19i[column1].dropna().unique() 
UCNets_a19f = UCNets_a19f[column2].dropna().unique() 
UCNets_e1b = UCNets_e1b[column3].dropna().unique() 

column1_responses = '; '.join(str(item) for item in UCNets_a19i) #what we will feed to the model
column2_responses = '; '.join(str(item) for item in UCNets_a19f)
column3_responses = '; '.join(str(item) for item in UCNets_e1b)

UCNets_a19i = pd.DataFrame(UCNets_a19i, columns=[column1])
UCNets_a19i[column1] = UCNets_a19i[column1].astype(str).str.lower()
UCNets_a19i[column1] = UCNets_a19i[column1].str.strip()
UCNets_a19i = UCNets_a19i[UCNets_a19i[column1] != ''].reset_index(drop=True) #trimming all empty rows
UCNets_a19i = UCNets_a19i.iloc[:175]

UCNets_a19f = pd.DataFrame(UCNets_a19f, columns=[column2])
UCNets_a19f[column2] = UCNets_a19f[column2].astype(str).str.lower()
UCNets_a19f[column2] = UCNets_a19f[column2].str.strip()
UCNets_a19f = UCNets_a19f[UCNets_a19f[column2] != ''].reset_index(drop=True)
UCNets_a19f = UCNets_a19f.iloc[:175]

UCNets_e1b = pd.DataFrame(UCNets_e1b, columns=[column3])
UCNets_e1b[column3] = UCNets_e1b[column3].astype(str).str.lower()
UCNets_e1b[column3] = UCNets_e1b[column3].str.strip()
UCNets_e1b = UCNets_e1b[UCNets_e1b[column3] != ''].reset_index(drop=True)
UCNets_e1b = UCNets_e1b.iloc[:175]

Here, I'm trying to "force" the model to "think" in steps by first A. trying to process the response into its own words and B. having it interact with that object. That is, instead of all steps being given at once, I'm having it think in steps. 

This time, I will have it think in a "chain," where I will have it output a response and then feed that response back to it in a seperate prompt. 

In [9]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following ways they took steps to make new friends and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append(pd.DataFrame({"1": ["e"]}))
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

First, I want to find out whether adjusting "creativity" makes a difference in how accurate the output is.

In [10]:
creativity = 0

a19i = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            GPT_4,
                            creativity,
                            categories = user_categories1)

a19i.to_csv('data/a19i_phase1_gpt.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to start l

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with my fiance" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "the rent was increasing; they wanted to renew the rent $400 extra, a month to re-lease.  made sense regarding my career.  and i wanted a

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "change of pace, see different part of the city" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "we were in a relationship and wanted to move in together.  we're both moving apart so  wanted to be closer togeth

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "i was told that she was really looking for someone. because i have a british colonial thing and she does, too, i had a real connection with her, and it's worked out really well. she offered me free rent for helping her out with her liviing situation." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not pre

{"1":1,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "lease was up and we wanted to find a bigger place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "employment." into all of the following ways they took steps to make new friends and select all that apply:   

...

{"1":1,"2":0,"3":1,"4":0,"5":1,"6":0}
Categorize this survey response "i was living in berkeley at that time as a student. i have since graduated and decided to move to hawaii with my best friend to adventure and take a year off before going back to grad school." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
C

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "it wasn't a great situation where i was living. scary dogs. i moved to due to environment." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "when i was renting in san jose, the owners were kinda mean and too ex

{"1":0,"2":1,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "i moved twice.  once because my dad bought a house in richmond, and then, my partner and i decided it would be a good idea to move in together, so we moved here." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "bought a home" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "this house is way more beautiful than the one before.  and i like this many people.  it is also academic themed, so i wanted to live there for the

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "for financial reasons. rent was raised." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "my work moved to the city" into all of the following ways they took steps to make new friends and select all that apply:

In [11]:
a19f = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            GPT_4,
                            creativity,
                            user_categories2)

a19f.to_csv('data/a19f_phase1_gpt.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format wher

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "didn't do anything" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number b

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "local bars, work co-workers, intramural sports, meetups, conferences, etc" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "trying to join groups, been to seminars downtown, visited business establishments. been busy with work." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, 

...

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "joined meetup (app downloaded for different activities in the area for people to meet new friends for activities), connect with people of a friend, through work, dating sites" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "most of our friends are still withing "striking distance" and we still see or have them over regularly. (weekly dinner, weekly d&d, a couple of "every couple of weeks" lunches, etc.) we joined a neighborhood web-site. introduced ourselves to f" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interacti

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "go to fitness classes, go to happy hours organized by fiance work" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Prov

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "hanging out with coworkers, playing ultimate frisbee multiple times a week with different groups of people" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gym

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "i joined college alumni group activites and took some continuing education courses at stanford. i also joined the morgan hill activities center for exercise." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exerc

In [12]:
e1b = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            GPT_4,
                            creativity,
                            user_categories3)

e1b.to_csv('data/e1b_phase1_gpt.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":0,"2":0,"3":1,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their o

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal 

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":1}
Categorize this survey response "probably don't have, because violent childhood with my brother colors present day. handling father's estate by half-sister and step-sister included deception and possible fraud, so i'd never ask for their help." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being part

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "probably don't have, because all my relatives are dead" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a fam

...

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":0}
Categorize this survey response "definitely have, because i think we're fortunate enough that we have close families on both sides. on both sides my parents and linda's parents are still married so there's a lot of family stability there. on both sides family values include helping each other out." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "5.0, because part of it is history, i had a short marriage and my folks were upset i was involved in that and i became very independent. a little estranged for a bit." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotion

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because my sister would certainly help me in any rough or bad situation that might arise" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refe

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":0}
Categorize this survey response "might have, because their life situations and . . . we don't have a lot of relatives close by that would be easy for them to do." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own f

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because because i've done it for them so i'm sure they would do it for me." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to fami

In [32]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following ways they took steps to make new friends and select all that apply: \
        {categories_str} \
        Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        print(prompt)
        try:
            message = client.messages.create(
                model=user_model,
                max_tokens=1024,
                temperature=creativity,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )

            reply = message.content
            reply = reply[0].text
            
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {response}")
            
        extracted_json = re.findall(r'\{[^}]+\}', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append(pd.DataFrame({"1": ["e"]}))
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [34]:
a19i_claude = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            claude,
                            creativity,
                            categories = user_categories1)

a19i_claude.to_csv('data/a19i_phase1_claude.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to 

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with my fiance" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "the rent was increasing; they wanted to renew the rent $400 extra, a month to re-lease.  made sense regarding my career.  and i w

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "change of pace, see different part of the city" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "we were in a relationship and wanted to move in together.  we're both moving apart so  wanted to be closer

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "i was told that she was really looking for someone. because i have a british colonial thing and she does, too, i had a real connection with her, and it's worked out really well. she offered me free rent for helping her out with her liviing situation." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is 

{"1":1,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "lease was up and we wanted to find a bigger place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "employment." into all of the following ways they took steps to make new friends and select all that ap

...

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i was living in a sublet and the sublet ended." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "the landlord wanted to move into the unit so she gave us notice to move out." into all of the following wa

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "because of my job" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "i had lived in an apartment near the cal campus for 9 years since my divorce. in 2015, i sold a previous home which was being used as a

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "wife retired, downsized, no mortgage now." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":1,"5":1,"6":1}
Categorize this survey response "we downsized in order to buy this new home and condos for each of our daughters." into all of the following way

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "to be with my wife and go to business school" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "change of job" into all of the following ways they took steps to make new friends and select all that apply:

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "family housing i. a better area of the city" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in proper JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "we bought a house in alameda" into all of the following ways they took steps to make new friends and select a

In [35]:
a19f_claude = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            claude,
                            creativity,
                            categories = user_categories2)

a19f_claude.to_csv('data/a19f_phase1_claude.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in proper JSON form

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "didn't do anything" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in proper JSON format where the n

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "local bars, work co-workers, intramural sports, meetups, conferences, etc" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "trying to join groups, been to seminars downtown, visited business establishments. been busy with work." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, 

...

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "joined meetup (app downloaded for different activities in the area for people to meet new friends for activities), connect with people of a friend, through work, dating sites" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "most of our friends are still withing "striking distance" and we still see or have them over regularly. (weekly dinner, weekly d&d, a couple of "every couple of weeks" lunches, etc.) we joined a neighborhood web-site. introduced ourselves to f" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interacti

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "just like talking to the other people in the co op, sharing meals with them, doing activities with them." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms,

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "met our neighbors and have get togethers in park/social gatherings" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Pro

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "just kept my online friends. too afraid to meet new people. also, couldn't drive." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clu

In [37]:
e1b_claude = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            claude,
                            creativity,
                            user_categories3)

e1b_claude.to_csv('data/e1b_phase1_claude.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":0,"2":0,"3":1,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their o

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal 

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":1}
Categorize this survey response "probably don't have, because violent childhood with my brother colors present day. handling father's estate by half-sister and step-sister included deception and possible fraud, so i'd never ask for their help." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being part

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "probably don't have, because all my relatives are dead" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a fam

...

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "definitely have, because i think we're fortunate enough that we have close families on both sides. on both sides my parents and linda's parents are still married so there's a lot of family stability there. on both sides family values include helping each other out." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "5.0, because part of it is history, i had a short marriage and my folks were upset i was involved in that and i became very independent. a little estranged for a bit." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotion

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because my sister would certainly help me in any rough or bad situation that might arise" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refe

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because well, if i had a life-threatening illness i would rely on my son because he's a doctor. plus i think he's worried about us so he'd be very supportive, do what he could. plus i might need to get some help with the trust and i can't do that unle" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again apply

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because well my parents are still in the house that they were raised in so they have lots of bedrooms cause it's just the two of them." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and 

In [68]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    LLAMA_API_KEY = os.getenv("PERPLEXITY_API_KEY")

    client = OpenAI(api_key=LLAMA_API_KEY, base_url="https://api.perplexity.ai")
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following ways they took steps to make new friends and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append(pd.DataFrame({"1": ["e"]}))
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [69]:
a19i_llama = extract_categories(question1, 
                            UCNets_a19i_head[column1], 
                            perplexity,
                            creativity,
                            categories = user_categories1)

a19i_llama.to_csv('data/a19i_phase1_llama.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to start l

In [70]:
a19i_llama = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            perplexity,
                            creativity,
                            categories = user_categories1)

a19i_llama.to_csv('data/a19i_phase1_llama.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to start l

{"1":1,//tostartlivingwithortostaywithpartner/spouse"2":0,//relationshipchange(divorce,breakup,etc)"3":0,//thepersonhadajoborschoolorcareerchange,includingtransferredandretired"4":0,//theperson'spartner'sjoborschoolorcareerchange,includingtransferredandretired"5":0,//financialreasons(rentistooexpensive,payraise,etc)"6":0//relatedspecificallyfeaturesofthehome,suchasabiggerorsmalleryard}
Categorize this survey response "moved in with my fiance" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JS

{"1":1,"2":0,"3":0,"4":0,"5":0,//Assumingnoexplicitfinancialreasonmentioned"6":0}
Categorize this survey response "change of pace, see different part of the city" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "we were in a relationship and wanted to move in together.  we're both

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "i was told that she was really looking for someone. because i have a british colonial thing and she does, too, i had a real connection with her, and it's worked out really well. she offered me free rent for helping her out with her liviing situation." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not pre

{"1":1,"2":0,"3":1,"4":1,"5":0,"6":0}
Categorize this survey response "lease was up and we wanted to find a bigger place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "employment." into all of the following ways they took steps to make new friends and select all that apply:   

...

{"5":1,//financialreasons(rentistooexpensive,payraise,etc)"6":1//relatedspecificallytofeaturesofthehome,suchasabiggerorsmalleryard}
Categorize this survey response "my mom got diagnosed with leukemia, and the following year we lost our home." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with someone" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with my parents" into all of the following ways they took steps to make new friends and select all that apply:         1. to star

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "for research - beginning fulbright fellowship in indonesia, taiwan, hong kong, philippines" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "to move closer to family after the baby was born" into all of the fol

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "we wanted a larger and better place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "better location to my spouse's job." into all of the following ways they took steps to make new friends and select all that

{"3":1,"4":0,"1":0,"2":0,"5":0,"6":0}
Categorize this survey response "my landlord didn't want to rent anymore." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "due to the divorce." into all of the following ways they took steps to make new friends and select all that apply:     

In [71]:
a19f_llama = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            perplexity,
                            creativity,
                            categories = user_categories2)

a19f_llama.to_csv('data/a19f_phase1_llama.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format wher

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "didn't do anything" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number b

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "local bars, work co-workers, intramural sports, meetups, conferences, etc" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "trying to join groups, been to seminars downtown, visited business establishments. been busy with work." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, 

...

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "joined meetup (app downloaded for different activities in the area for people to meet new friends for activities), connect with people of a friend, through work, dating sites" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement

{"1":1,//Engagedwithlocalreligiousinstitutionssuchaschurches,synagogues,mosques,orotherformsofreligiouscommunities."2":0,//Frequentedlocalestablishmentslikebars,cafes,shops,ormallstointeractwithindividualspresentinthevicinity."3":0,//Directinvolvementinsecularvolunteeringefforts,contributingthroughactionandserviceratherthanmeremembershipinvolunteergroups."4":0,//Utilizingdigitalplatformssuchasonlinechats,internetnetworkingwebsites,ordatingappstoestablishconnectionsandfriendships."5":0,//Engagedininformal,non-professionalinteractionsandoutingswithcolleaguestofosterfriendships."6":1//Involvementinsports,exercise,oroutdoorrecreationalactivitiesthroughgyms,teams,orathleticclubs.}
Categorize this survey response "most of our friends are still withing "striking distance" and we still see or have them over regularly. (weekly dinner, weekly d&d, a couple of "every couple of weeks" lunches, etc.) we joined a neighborhood web-site. introduced ourselves to f" into all of the following ways they t

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "just like talking to the other people in the co op, sharing meals with them, doing activities with them." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms,

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "met our neighbors and have get togethers in park/social gatherings" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Pro

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "i joined college alumni group activites and took some continuing education courses at stanford. i also joined the morgan hill activities center for exercise." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exerc

In [72]:
e1b_llama = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            perplexity,
                            creativity,
                            user_categories3)

e1b_llama.to_csv('data/e1b_phase1_llama.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":0,"2":1,"3":1,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their o

{"1":0,//Asimplestatementwithoutfurtherelaboration"2":0,//Brieflyindicatinganimplicitdutyorobligation"3":0,//Explicitlyreferringtotheprincipleoffamilyobligations"4":1,//Theirownfamilybeingparticularlyclose,loyalty,aswellasemotionalsupportandlove"5":0,//Mentionofafamilymember'spersonalcharacter"6":1//Pastinstancesofreceivinghelporsupportfromfamilymembers}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations 

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "definitely have, because they're rich and said they love me. they've helped me before." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families 

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "definitely have, because very close family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member'

...

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because i am sure my children would try to help me. my sister might also want to help." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refere

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "definitely have, because ever since my grandfather died it's harder to know if our relatives would help. they might but not sure." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own 

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":0}
Categorize this survey response "definitely have, because we've always had each others backs" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of 

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because we are close- children are very caring" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}


In [75]:
a19i_llama_8B = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            perplexity_light,
                            creativity,
                            categories = user_categories1)

a19i_llama_8B.to_csv('data/a19i_phase1_llama_8B.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,//tostartlivingwithortostaywithpartner/spouse"2":0,//relationshipchange(divorce,breakup,etc)"3":0,//thepersonhadajoborschoolorcareerchange,includingtransferredandretired"4":0,//theperson'spartner'sjoborschoolorcar

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "i moved to be closer to my boyfriend, in the same house." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with my fiance" into all of the following ways they took steps to make new friends and select 

{"1":0,//tostartlivingwithortostaywithpartner/spouse"2":0,//relationshipchange(divorce,breakup,etc)"3":0,//thepersonhadajoborschoolorcareerchange,includingtransferredandretired"4":0,//theperson'spartner'sjoborschoolorcareerchange,includingtransferredandretired"5":1,//financialreasons(rentistooexpensive,payraise,etc)"6":0//relatedspecificallyfeaturesofthehome,suchasabiggerorsmalleryard}
Categorize this survey response "to move in with shamus" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSO

{"1":0,//tostartlivingwithortostaywithpartner/spouse"2":0,//relationshipchange(divorce,breakup,etc)"3":0,//thepersonhadajoborschoolorcareerchange,includingtransferredandretired"4":0,//theperson'spartner'sjoborschoolorcareerchange,includingtransferredandretired"5":1,//financialreasons(rentistooexpensive,payraise,etc)"6":0//relatedspecificallyfeaturesofthehome,suchasabiggerorsmalleryard}
Categorize this survey response "we got the house then found out i was pregnant within a two week period." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigge

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "job opportunity with the same company opened up in los angeles. had to relocate from bay area." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "the apartment complex was too loud. moved to a rental property my

...

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "spouse job" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "because my sister passed away and i was taking care of her. housing being so bad, i found a room in a teacher's house." into all of the following way

{"1":0,//tostartlivingwithortostaywithpartner/spouse"2":0,//relationshipchange(divorce,breakup,etc)"3":0,//thepersonhadajoborschoolorcareerchange,includingtransferredandretired"4":0,//theperson'spartner'sjoborschoolorcareerchange,includingtransferredandretired"5":1,//financialreasons(rentistooexpensive,payraise,etc)"6":0//relatedspecificallyfeaturesofthehome,suchasabiggerorsmalleryard}
Categorize this survey response "moved in with my parents" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in J

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "to move closer to family after the baby was born" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "i sold my home and moved back in with my parents. i owned my home with my sister and we no longer wanted to liv

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "we wanted a larger and better place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "better location to my spouse's job." into all of the following ways they took steps to make new friends and select all that

{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "my landlord didn't want to rent anymore." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "due to the divorce." into all of the following ways they took steps to make new friends and select all that apply:     

In [76]:
a19f_llama_8B = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            perplexity_light,
                            creativity,
                            categories = user_categories2)

a19f_llama_8B.to_csv('data/a19f_phase1_llama_8B.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format wher

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "didn't do anything" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number b

{"1":0,"2":1,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "local bars, work co-workers, intramural sports, meetups, conferences, etc" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "trying to join groups, been to seminars downtown, visited business establishments. been busy with work." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, 

...

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "joined meetup (app downloaded for different activities in the area for people to meet new friends for activities), connect with people of a friend, through work, dating sites" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "most of our friends are still withing "striking distance" and we still see or have them over regularly. (weekly dinner, weekly d&d, a couple of "every couple of weeks" lunches, etc.) we joined a neighborhood web-site. introduced ourselves to f" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interacti

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "just like talking to the other people in the co op, sharing meals with them, doing activities with them." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms,

{"1":0,"2":1,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "met our neighbors and have get togethers in park/social gatherings" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Pro

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "i joined college alumni group activites and took some continuing education courses at stanford. i also joined the morgan hill activities center for exercise." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exerc

In [77]:
e1b_llama_8B = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            perplexity_light,
                            creativity,
                            user_categories3)

e1b_llama_8B.to_csv('data/e1b_phase1_llama_8B.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":0,"2":1,"3":1,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their o

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal 

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":1}
Categorize this survey response "probably don't have, because violent childhood with my brother colors present day. handling father's estate by half-sister and step-sister included deception and possible fraud, so i'd never ask for their help." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being part

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "probably don't have, because all my relatives are dead" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a fam

...

{"1":0,"2":1,"3":1,"4":1,"5":1,"6":0}
Categorize this survey response "definitely have, because i think we're fortunate enough that we have close families on both sides. on both sides my parents and linda's parents are still married so there's a lot of family stability there. on both sides family values include helping each other out." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "5.0, because part of it is history, i had a short marriage and my folks were upset i was involved in that and i became very independent. a little estranged for a bit." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotion

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because my sister would certainly help me in any rough or bad situation that might arise" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refe

{"1":0,"2":1,"3":1,"4":1,"5":0,"6":0}
Categorize this survey response "might have, because their life situations and . . . we don't have a lot of relatives close by that would be easy for them to do." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own f

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because because i've done it for them so i'm sure they would do it for me." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to fami

In [90]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    API_KEY = os.environ["MISTRAL_API_KEY"]

    client = Mistral(api_key=API_KEY)
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following ways they took steps to make new friends and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        print(prompt)
        try:
            response = client.chat.complete(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append(pd.DataFrame({"1": ["e"]}))
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [92]:
a19i_mistral = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            mistral,
                            creativity,
                            categories = user_categories1)

a19i_mistral.to_csv('data/a19i_phase1_mistral.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to start l

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "moved in with my fiance" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "the rent was increasing; they wanted to renew the rent $400 extra, a month to re-lease.  made sense regarding my career.  and i wanted a

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "change of pace, see different part of the city" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "we were in a relationship and wanted to move in together.  we're both moving apart so  wanted to be closer togeth

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "i was told that she was really looking for someone. because i have a british colonial thing and she does, too, i had a real connection with her, and it's worked out really well. she offered me free rent for helping her out with her liviing situation." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not pre

{"1":1,"2":0,"3":1,"4":0,"5":0,"6":0}
Categorize this survey response "lease was up and we wanted to find a bigger place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "employment." into all of the following ways they took steps to make new friends and select all that apply:   

...

{"1":1,"2":0,"3":1,"4":0,"5":1,"6":0}
Categorize this survey response "i was living in berkeley at that time as a student. i have since graduated and decided to move to hawaii with my best friend to adventure and take a year off before going back to grad school." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":1,"4":0,"5":0,"6":0}
C

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "it wasn't a great situation where i was living. scary dogs. i moved to due to environment." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "when i was renting in san jose, the owners were kinda mean and too ex

{"1":0,"2":1,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "i moved twice.  once because my dad bought a house in richmond, and then, my partner and i decided it would be a good idea to move in together, so we moved here." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":1,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "bought a home" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":1}
Categorize this survey response "this house is way more beautiful than the one before.  and i like this many people.  it is also academic themed, so i wanted to live there for the

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "for financial reasons. rent was raised." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "my work moved to the city" into all of the following ways they took steps to make new friends and select all that apply:

In [93]:
a19f_mistral = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            mistral,
                            creativity,
                            categories = user_categories2)

a19f_mistral.to_csv('data/a19f_phase1_mistral.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format wher

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "didn't do anything" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number b

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "local bars, work co-workers, intramural sports, meetups, conferences, etc" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "trying to join groups, been to seminars downtown, visited business establishments. been busy with work." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, 

...

{"1":0,"2":1,"3":0,"4":0,"5":0,"6":0}
Categorize this survey response "joined meetup (app downloaded for different activities in the area for people to meet new friends for activities), connect with people of a friend, through work, dating sites" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "most of our friends are still withing "striking distance" and we still see or have them over regularly. (weekly dinner, weekly d&d, a couple of "every couple of weeks" lunches, etc.) we joined a neighborhood web-site. introduced ourselves to f" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interacti

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "go to fitness classes, go to happy hours organized by fiance work" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Prov

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":1}
Categorize this survey response "hanging out with coworkers, playing ultimate frisbee multiple times a week with different groups of people" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gym

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "i joined college alumni group activites and took some continuing education courses at stanford. i also joined the morgan hill activities center for exercise." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exerc

In [94]:
e1b_mistral = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            mistral,
                            creativity,
                            categories = user_categories3)

e1b_mistral.to_csv('data/e1b_phase1_mistral.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their o

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal 

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":1}
Categorize this survey response "probably don't have, because violent childhood with my brother colors present day. handling father's estate by half-sister and step-sister included deception and possible fraud, so i'd never ask for their help." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being part

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "probably don't have, because all my relatives are dead" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a fam

...

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":0}
Categorize this survey response "definitely have, because i think we're fortunate enough that we have close families on both sides. on both sides my parents and linda's parents are still married so there's a lot of family stability there. on both sides family values include helping each other out." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0}
Categorize this survey response "5.0, because part of it is history, i had a short marriage and my folks were upset i was involved in that and i became very independent. a little estranged for a bit." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotion

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because my sister would certainly help me in any rough or bad situation that might arise" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refe

{"1":0,"2":0,"3":0,"4":1,"5":1,"6":0}
Categorize this survey response "might have, because their life situations and . . . we don't have a lot of relatives close by that would be easy for them to do." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own f

{"1":0,"2":0,"3":0,"4":1,"5":0,"6":0}
Categorize this survey response "definitely have, because because i've done it for them so i'm sure they would do it for me." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to fami

In [96]:
a19i_mistral_8B = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            mistral_light,
                            creativity,
                            categories = user_categories1)

a19i_mistral_8B.to_csv('data/a19i_phase1_mistral_8B.csv')

Categorize this survey response "relocated back to east coast - closer to my sons" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "move in together with my partner" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with p

{"1":"e"}
Categorize this survey response "the rent was increasing; they wanted to renew the rent $400 extra, a month to re-lease.  made sense regarding my career.  and i wanted a backyard for my dog." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "i was paying rent that rose to $9000/mo and it made econom

{"1":"e"}
Categorize this survey response "rent increase" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "to buy a home" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorc

{"1":"e"}
Categorize this survey response "for my son's school." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "move #1  - 60 day notice from landlord
move #2 - better opportunity for a nicer place
move #3 - nicer place was short term
move #4 - found semi-ideal place to live" into all of the following ways

{"1":"e"}
Categorize this survey response "we missed colorado (both my wife and i are from there originally)." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "iwanted to move to a house with a yard" into all of the following ways they took steps to make new friends and select all that apply:         1. to s

...

{"1":"e"}
Categorize this survey response "because i bought this place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "not enough space and expensive." into all of the following ways they took steps to make new fr

{"1":"e"}
Categorize this survey response "for graduate school" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "for a change of environment." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relati

{"1":"e"}
Categorize this survey response "roommates graduated, and rent increased" into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "for research - beginning fulbright fellowship in indonesia, taiwan, hong kong, philippines" into all of the following ways they took steps to make new friends and select all th

{"1":"e"}
Categorize this survey response "graduated from college and got a job." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
{"1":"e"}
Categorize this survey response "we wanted a larger and better place." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay wit

{"1":"e"}
Categorize this survey response "my landlord didn't want to rent anymore." into all of the following ways they took steps to make new friends and select all that apply:         1. to start living with or to stay with partner/spouse
2. relationship change (divorce, breakup, etc)
3. the person had a job or school or career change, including transferred and retired
4. the person's partner's job or school or career change, including transferred and retired
5. financial reasons (rent is too expensive, pay raise, etc)
6. related specifically features of the home, such as a bigger or smaller yard         Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values.
An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "due to the divorce." into all of the following ways they took steps to make new fr

In [97]:
a19f_mistral_8B = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            mistral_light,
                            creativity,
                            categories = user_categories2)

a19f_mistral_8B.to_csv('data/a19f_phase1_mistral_8B.csv')

Categorize this survey response "volunteering, joining an exercise group, chatting up other dog owners." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format wher

{"1":"e"}
Categorize this survey response "i've initiated some social gatherings to get to know other friends of friends." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work i

{"1":"e"}
Categorize this survey response "nothing" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number belonging to each category is the key an

{"1":"e"}
Categorize this survey response "nothing yet. just moved here. mostly just us two going out together to meet new friends before i head out." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":"e"}
Categorize this survey response "getting involved in local synagogue, reach out to people i know that are local, hang out at the coffee shop, become involved in my son's school." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities t

...

{"1":"e"}
Categorize this survey response "biggest thing was finding a church for me. groups on campus, bible study, started meeting other classmates" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.

{"1":"e"}
Categorize this survey response "picked up a second job" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number belonging to each categor

{"1":"e"}
Categorize this survey response "no steps" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.         Provide your work in JSON format where the number belonging to each category is the key a

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "not much" into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational activities through gyms, teams, or athletic clubs.     

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "we have met our neighbors. we already had friends in the area." into all of the following ways they took steps to make new friends and select all that apply:         1. Engaged with local religious institutions such as churches, synagogues, mosques, or other forms of religious communities.
2. Frequented local establishments like bars, cafes, shops, or malls to interact with individuals present in the vicinity.
3. Direct involvement in secular volunteering efforts, contributing through action and service rather than mere membership in volunteer groups.
4. Utilizing digital platforms such as online chats, internet networking websites, or dating apps to establish connections and friendships.
5. Engaged in informal, non-professional interactions and outings with colleagues to foster friendships.
6. Involvement in sports, exercise, or outdoor recreational a

In [98]:
e1b_mistral_8B = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            mistral_light,
                            creativity,
                            categories = user_categories3)

e1b_mistral_8B.to_csv('data/e1b_phase1_mistral_8B.csv')

Categorize this survey response "probably don't have, because not many relatives" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such as 'the

{"1":"e"}
Categorize this survey response "definitely have, because i think we're very close. that's the kind of thing that goes without saying for us, at least for my family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than refer

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "definitely have, because parents" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references t

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "definitely have, because they're rich and said they love me. they've helped me before." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and 

{"1":"e"}
Categorize this survey response "definitely have, because very close family." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal character, such a

...

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "definitely have, because my two brothers and my son" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rathe

{"1":"e"}
Categorize this survey response "definitely have, because because of being siblings, obligation." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's person

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "definitely have, because they're very close physically. they have space, they own homes. they're not too poor. i do talk to them regularly." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particular

{"1":"e"}
Categorize this survey response "definitely have, because shared history for a long time" into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).
5. Mention of a family member's personal chara

An error occurred: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
{"1":"e"}
Categorize this survey response "definitely have, because i have a very close relationship with my relatives." into all of the following ways they took steps to make new friends and select all that apply:         1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.
4. Their own family being particularly close, loyalty,  as well as emotional support and love withi

In [143]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories,
                       user_api_key):
    
    API_KEY = user_api_key
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{user_model}:generateContent"
    headers = {"Content-Type": "application/json"}
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    max_retries = 3
    for survey_response in survey_input:
        prompt = f"""Categorize this survey response "{survey_response}" into all of the following ways they took steps to make new friends and select all that apply: \
        {categories_str} \
        Provide only a JSON in proper format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        
        attempt = 0
        success = False
        reply = """{"1":"e"}"""
        
        while attempt < max_retries and not success:
            attempt += 1
            time.sleep(2)
            try:
                api_response = requests.post(
                    f"{url}?key={API_KEY}",
                    headers=headers,
                    json={
                        "contents": [{
                            "parts": [{"text": prompt}]
                        }],
                        "generationConfig": {
                            "temperature": creativity
                        }
                    }
                )
                if api_response.status_code == 200:
                    success = True
                    reply = api_response.json()['candidates'][0]['content']['parts'][0]['text']
                    print(reply)
                else: 
                    print(f"Failed with status code: {api_response.status_code}")
                    if attempt < max_retries:
                        print(f"Retrying... ({attempt}/{max_retries})")
                        time.sleep(2)
            
            except Exception as e:
                print(f"An error occurred: {e}")
        
        link1.append(reply)
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append(pd.DataFrame({"1": ["e"]}))
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [209]:
a19i_google = extract_categories(question1, 
                            UCNets_a19i[column1], 
                            google,
                            creativity,
                            categories = user_categories1,
                            user_api_key = os.environ["GOOGLE_API_KEY"])

a19i_google.to_csv('data/a19i_phase1_google.csv')

```json
{
  "1": 0,
  "2": 0,
  "3": 0,
  "4": 0,
  "5": 0,
  "6": 0
}
```

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0}
```json
{
  "1": 1,
  "2": 0,
  "3": 0,
  "4": 0,
  "5": 0,
  "6": 0
}
```

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
```json
{
  "1": 1,
  "2": 0,
  "3": 0,
  "4": 0,
  "5": 0,
  "6": 0
}
```

{"1":1,"2":0,"3":0,"4":0,"5":0,"6":0}
Failed with status code: 429
Retrying... (1/3)
Failed with status code: 429
Retrying... (2/3)
Failed with status code: 429
{"1":"e"}
Failed with status code: 429
Retrying... (1/3)
Failed with status code: 429
Retrying... (2/3)
Failed with status code: 429
{"1":"e"}
Failed with status code: 429
Retrying... (1/3)
Failed with status code: 429
Retrying... (2/3)
Failed with status code: 429
{"1":"e"}
Failed with status code: 503
Retrying... (1/3)
Failed with status code: 429
Retrying... (2/3)
Failed with status code: 429
{"1":"e"}
Failed with status code: 429
Retrying... (1/3)
Failed with status code: 429
Retrying... (2/3)
Failed with status code: 429
{"1

KeyboardInterrupt: 

In [ ]:
a19f_google = extract_categories(question2, 
                            UCNets_a19f[column2], 
                            google,
                            creativity,
                            categories = user_categories2,
                            user_api_key = os.environ["GOOGLE_API_KEY"])

a19f_google.to_csv('data/a19f_phase1_google.csv')

In [ ]:
e1b_google = extract_categories(question3, 
                            UCNets_e1b[column3], 
                            google,
                            creativity,
                            categories = user_categories3)

e1b_google.to_csv('data/e1b_phase1_google.csv')

I'm giving up on the google calls. I got locked out of making API calls.

Next, I want to compare how similar these categorizations are to each other. First I'm going to prepare the data.

In [154]:
from scipy import stats
from scipy.stats import chi2_contingency

In [192]:
Main = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Chris").drop(["Response"], axis=1)
Main = Main.iloc[:175, :8]
Main = Main.filter(['Partner or spouse','Relationship end','Own job/school/career','Partner’s job/school/career','Financial','Housing, concretely'], axis=1)
Main.columns = range(Main.shape[1])

Claude = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Claude").drop(["Response","prim_key"], axis=1)
Claude = Claude.iloc[:175, :8]
Claude = Claude.filter(['Partner or spouse','Relationship end','Own job/school/career','Partner’s job/school/career','Financial','Housing, concretely'], axis=1)
Claude.columns = range(Claude.shape[1])

Rebeca = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Rebeca").drop(["Response"], axis=1)
Rebeca = Rebeca.iloc[:175, :8]
Rebeca = Rebeca.filter(['Partner or spouse','Relationship end','Own job/school/career','Partner’s job/school/career','Financial','Housing, concretely'], axis=1)
Rebeca.columns = range(Rebeca.shape[1])

a19i_llama = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_llama.csv")
a19i_llama = a19i_llama.drop(columns=['Unnamed: 0'])
a19i = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_gpt.csv")
a19i_claude = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_claude.csv")
a19i_mistral = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_mistral.csv")
a19i_llama = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_llama.csv")

#printing row count to make sure we're all on the same level
print(f"The dataframe Main has {len(Main)} rows.")

print(f"The dataframe from openai has {len(a19i)} rows.")

print(f"The dataframe from claude has {len(a19i_claude)} rows.")

print(f"The dataframe from llama 3.1 has {len(a19i_llama)} rows.")

print(f"The dataframe from mistral has {len(a19i_mistral)} rows.")

#filling in missing with 0
Main.fillna(0, inplace=True)
Claude.fillna(0, inplace=True)
Rebeca.fillna(0, inplace=True)

#making all columns numeric
Main = Main.astype(int, errors='ignore')
Claude = Claude.astype(int, errors='ignore')
Rebeca = Rebeca.astype(int, errors='ignore')
a19i = a19i.astype(int, errors='ignore')
a19i_claude = a19i_claude.astype(int, errors='ignore')
a19i_llama = a19i_llama.astype(int, errors='ignore')
a19i_mistral = a19i_mistral.astype(int, errors='ignore')

Main.to_csv('data/a19i_phase1_chris.csv', index=False)
Claude.to_csv('data/a19i_phase1_claudehuman.csv', index=False)
Rebeca.to_csv('data/a19i_phase1_rebeca.csv', index=False)

The dataframe Main has 175 rows.
The dataframe from openai has 175 rows.
The dataframe from claude has 175 rows.
The dataframe from llama 3.1 has 175 rows.
The dataframe from mistral has 175 rows.


In [195]:
a19i = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_gpt.csv")
a19i_claude = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_claude.csv")
a19i_mistral = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_mistral.csv")
a19i_llama = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19i_phase1_llama.csv")

In [197]:
a19i = a19i.drop(columns=a19i.columns[:4])
a19i_claude = a19i_claude.drop(columns=a19i_claude.columns[:4])
a19i_mistral = a19i_mistral.drop(columns=a19i_mistral.columns[:4])
#had to make many manual fixes
a19i_llama = a19i_llama.drop(columns=a19i_llama.columns[:4])

Now, let's run some overall classification comparisons

In [213]:
category_dfs = {
    'Claude': Claude,
    'Rebeca': Rebeca,
    'a19i': a19i,
    'a19i_claude': a19i_claude,
    'a19i_llama': a19i_llama,
    'a19i_mistral': a19i_mistral
}
result_dfs = {}

for cat_name, cat_df in category_dfs.items():
    association_measures = []
    p_values = []
    jaccard_similarities = []
    cosine_similarities = []  # New list to store cosine similarities

    for i in range(6):
        main_col = Main.columns[i]
        cat_col = cat_df.columns[i]

        # Create a contingency table
        contingency_table = pd.crosstab(Main[main_col], cat_df[cat_col])

        # Calculate the Chi-squared test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        association_measures.append(chi2)
        p_values.append(p_value)

        # Calculate Jaccard Similarity
        intersection = sum((Main[main_col] == 1) & (cat_df[cat_col] == 1))
        union = sum((Main[main_col] == 1) | (cat_df[cat_col] == 1))
        jaccard_similarity = intersection / union if union != 0 else 0
        jaccard_similarities.append(jaccard_similarity)

        # Calculate Cosine Similarity
        dot_product = np.dot(Main[main_col], cat_df[cat_col])
        magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
        magnitude_cat = np.sqrt(np.dot(cat_df[cat_col], cat_df[cat_col]))
        cosine_similarity = dot_product / (magnitude_main * magnitude_cat) if magnitude_main != 0 and magnitude_cat != 0 else 0
        cosine_similarities.append(cosine_similarity)  # Append to the list

    # Create the DataFrame
    chi_squared_jaccard_cosine_df = pd.DataFrame({
        'Chi2_Main_Cat': association_measures,
        'P_value_Main_Cat': p_values,
        'Jaccard_Similarity': jaccard_similarities,
        'Cosine_Similarity': cosine_similarities,  # Corrected column name
    }, index=Main.columns[:6])

    # Store the DataFrame in a dictionary with a unique key
    result_dfs[cat_name] = chi_squared_jaccard_cosine_df

unified_df = pd.DataFrame()

for cat_name, df in result_dfs.items():
    prefixed_df = df.add_prefix(f'{cat_name}_')
    unified_df = pd.concat([unified_df, prefixed_df], axis=1)
    
unified_df.to_csv('data/a19i_phase1_comparisons.csv', index=False)
result_dfs

{'Claude':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity  Cosine_Similarity
 0     118.445675      1.384958e-27            0.760000           0.867227
 1     118.906525      1.097829e-27            0.800000           0.888889
 2     156.955970      5.233244e-36            0.944444           0.971825
 3     152.827752      4.177510e-35            1.000000           1.000000
 4     120.980333      3.859391e-28            0.785714           0.880705
 5      95.004118      1.899896e-22            0.666667           0.800853,
 'Rebeca':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity  Cosine_Similarity
 0     155.514409      1.080888e-35            0.950000           0.974679
 1      99.537769      1.924579e-23            0.700000           0.824958
 2     128.449845      8.948005e-30            0.805556           0.897527
 3     130.279870      3.558895e-30            0.875000           0.935414
 4     126.211811      2.763555e-29            0.809524           0.895047
 5  

First, I want to produce a column-wise average comparison. This measure tells us about individual-categorizations more than the overall "correctness" of a categorization. 

In [219]:
# Define the columns you want to compare and their corresponding names
columns_to_compare = [
    ('Claude_Cosine_Similarity', 'Claude'),
    ('Rebeca_Cosine_Similarity', 'Rebeca'),
    ('a19i_Cosine_Similarity', 'GPT'),
    ('a19i_claude_Cosine_Similarity', 'Claude'),
    ('a19i_llama_Cosine_Similarity', 'Llama'),
    ('a19i_mistral_Cosine_Similarity', 'Mistral')
]

# Create the comparison table
table_a19i_column_comparisons = pd.DataFrame({
    'Coder/Model': ['Coder 1'] + [name for _, name in columns_to_compare],
    'a19i Average Cosine Similarity': [1] + [unified_df[col].mean() for col, _ in columns_to_compare]
})

# Set 'Coder/Model' as the index
table_a19i_column_comparisons.set_index('Coder/Model', inplace=True)

print(table_a19i_column_comparisons)


             a19i Average Cosine Similarity
Coder/Model                                
Coder 1                            1.000000
Claude                             0.901583
Rebeca                             0.899129
GPT                                0.853491
Claude                             0.803038
Llama                              0.766499
Mistral                            0.815867


Now, let's repeat the steps above for a19f

notes:
llama: 10 errors
mistral: 0 errors
claude: 0 errors
gpt: 0 errors

In [297]:
Main = pd.read_csv("../UCNets_Classification/Hand_Coding_Surveys/a19fg/Master_Codes_a19f.csv")
Main = Main.filter(['Religious Community Involvement','Local Venue Visitation','Secular Volunteer Service','Online Social Networking','Workplace Socializing','Athletic Engagement'], axis=1)
Main.columns = range(Main.shape[1])
Main = Main.iloc[:175]

# Assuming Main_a19f is your DataFrame

print(f"The dataframe Main has {len(Main)} rows.")

Nigel = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19fg/a19fg_Master.xlsx", engine='openpyxl', sheet_name="Nigel_a19f")
Nigel = Nigel.filter(['Religious Community Involvement','Local Venue Visitation','Secular Volunteer Service','Online Social Networking','Workplace Socializing','Athletic Engagement'], axis=1)
Nigel.columns = range(Nigel.shape[1])
Nigel = Nigel.iloc[:175]

Rebeca = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19fg/a19fg_Master.xlsx", engine='openpyxl', sheet_name="Rebeca_a19f")
Rebeca = Rebeca.iloc[:175]
Rebeca = Rebeca.filter(['Religious Community Involvement','Local Venue Visitation','Secular Volunteer Service','Online Social Networking','Workplace Socializing','Athletic Engagement'], axis=1)
Rebeca.columns = range(Rebeca.shape[1])

#had to make fixes
a19f_llama = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19f_phase1_llama.csv")
a19f_llama = a19f_llama.drop(columns=['Unnamed: 0'])

a19f = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19f_phase1_gpt.csv")
a19f_claude = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19f_phase1_claude.csv")
a19f_mistral = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19f_phase1_mistral.csv")
a19f_llama = pd.read_csv("/Users/chrissoria/Documents/Research/Categorization_AI_experiments/data/a19f_phase1_llama.csv")

#filling in missing with 0
Main_a19f.fillna(0, inplace=True)
Nigel.fillna(0, inplace=True)
Rebeca.fillna(0, inplace=True)

#making all columns numeric
Main_a19f = Main_a19f.astype(int, errors='ignore')
Nigel = Nigel.astype(int, errors='ignore')
Rebeca = Rebeca.astype(int, errors='ignore')

Main_a19f.to_csv('data/a19f_phase1_chris.csv', index=False)
Nigel.to_csv('data/a19f_phase1_nigel.csv', index=False)
Rebeca.to_csv('data/a19f_phase1_rebeca.csv', index=False)

The dataframe Main has 175 rows.


In [298]:
a19f.fillna(0, inplace=True)
a19f_claude.fillna(0, inplace=True)
a19f_mistral.fillna(0, inplace=True)
a19f_llama.fillna(0, inplace=True)

a19f = a19f.astype(int, errors='ignore')
a19f_claude = a19f_claude.astype(int, errors='ignore')
a19f_llama = a19f_llama.astype(int, errors='ignore')
a19f_mistral = a19f_mistral.astype(int, errors='ignore')

In [299]:
a19f = a19f.drop(columns=a19f.columns[:4])
a19f_claude = a19f_claude.drop(columns=a19f_claude.columns[:4])
a19f_mistral = a19f_mistral.drop(columns=a19f_mistral.columns[:4])
#had to make many manual fixes
a19f_llama = a19f_llama.drop(columns=a19f_llama.columns[:4])

a19f_claude

,1,2,3,4,5,6
0,0,0,1,0,0,1
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,1,1,0
...,...,...,...,...,...,...
170,0,0,0,1,0,0
171,0,0,0,0,0,1
172,0,0,0,1,0,0
173,0,1,0,0,0,0


In [300]:
category_dfs = {
    'Nigel': Nigel,
    'Rebeca': Rebeca,
    'a19f': a19f,
    'a19f_claude': a19f_claude,
    'a19f_llama': a19f_llama,
    'a19f_mistral': a19f_mistral
}
result_dfs = {}


for cat_name, cat_df in category_dfs.items():
    association_measures = []
    p_values = []
    jaccard_similarities = []
    cosine_similarities = []  # New list to store cosine similarities

    for i in range(6):
        main_col = Main.columns[i]
        cat_col = cat_df.columns[i]

        # Create a contingency table
        contingency_table = pd.crosstab(Main[main_col], cat_df[cat_col])

        # Calculate the Chi-squared test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        association_measures.append(chi2)
        p_values.append(p_value)

        # Calculate Jaccard Similarity
        intersection = sum((Main[main_col] == 1) & (cat_df[cat_col] == 1))
        union = sum((Main[main_col] == 1) | (cat_df[cat_col] == 1))
        jaccard_similarity = intersection / union if union != 0 else 0
        jaccard_similarities.append(jaccard_similarity)

        # Calculate Cosine Similarity
        dot_product = np.dot(Main[main_col], cat_df[cat_col])
        magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
        magnitude_cat = np.sqrt(np.dot(cat_df[cat_col], cat_df[cat_col]))
        cosine_similarity = dot_product / (magnitude_main * magnitude_cat) if magnitude_main != 0 and magnitude_cat != 0 else 0
        cosine_similarities.append(cosine_similarity)  # Append to the list

    # Create the DataFrame
    chi_squared_jaccard_cosine_df = pd.DataFrame({
        'Chi2_Main_Cat': association_measures,
        'P_value_Main_Cat': p_values,
        'Jaccard_Similarity': jaccard_similarities,
        'Cosine_Similarity': cosine_similarities,  # Corrected column name
    }, index=Main.columns[:6])

    # Store the DataFrame in a dictionary with a unique key
    result_dfs[cat_name] = chi_squared_jaccard_cosine_df

unified_df = pd.DataFrame()

for cat_name, df in result_dfs.items():
    prefixed_df = df.add_prefix(f'{cat_name}_')
    unified_df = pd.concat([unified_df, prefixed_df], axis=1)
    
unified_df.to_csv('data/a19f_phase1_comparisons.csv', index=False)
result_dfs

{'Nigel':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity  Cosine_Similarity
 0     111.921952      3.716567e-26            0.736842           0.848875
 1      90.049683      2.322539e-21            0.631579           0.774597
 2      64.596525      9.191735e-16            0.450000           0.670820
 3     101.037860      9.024197e-24            0.687500           0.815374
 4      80.372990      3.100038e-19            0.606061           0.774839
 5     134.628201      3.981635e-31            0.842105           0.917663,
 'Rebeca':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity  Cosine_Similarity
 0     142.466279      7.690030e-33            0.888889           0.941176
 1     137.456443      9.582222e-32            0.866667           0.930949
 2     134.919440      3.438407e-31            0.888889           0.942809
 3     110.633877      7.117283e-26            0.733333           0.848668
 4     142.806237      6.480320e-33            0.878049           0.937043
 5   

In [302]:
# Define the columns you want to compare and their corresponding names
columns_to_compare = [
    ('Nigel_Cosine_Similarity', 'Nigel'),
    ('Rebeca_Cosine_Similarity', 'Rebeca'),
    ('a19f_Cosine_Similarity', 'GPT'),
    ('a19f_claude_Cosine_Similarity', 'Claude'),
    ('a19f_llama_Cosine_Similarity', 'Llama'),
    ('a19f_mistral_Cosine_Similarity', 'Mistral')
]

# Create the comparison table
table_a19f_column_comparisons = pd.DataFrame({
    'Coder/Model': ['Coder 1'] + [name for _, name in columns_to_compare],
    'a19f Average Cosine Similarity': [1] + [unified_df[col].mean() for col, _ in columns_to_compare]
})

# Set 'Coder/Model' as the index
table_a19f_column_comparisons.set_index('Coder/Model', inplace=True)

print(table_a19f_column_comparisons)


             a19f Average Cosine Similarity
Coder/Model                                
Coder 1                            1.000000
Nigel                              0.800361
Rebeca                             0.904636
GPT                                0.763123
Claude                             0.737941
Llama                              0.698380
Mistral                            0.708737
